# Sistemas de Defensas

## Dimensionamento

##### PIANC
$
E_A = 0,5 \cdot \frac{w}{g} \cdot V_A^2 \cdot C_m \cdot C_e \cdot C_c \cdot C_a \cdot F_s
$

$w$ Deslocamento do navio;
$g$ Aceleração da Gravidade;
$V_A$ Velocidade de atracação perpendicular a linha de atracação;
$C_m$ Coeficiente de Massa;
$C_e$ Coeficiente de excentricidade;
$C_c$ Coeficiente de configuração do cais;
$C_a$ Coeficiente de amortecimento;
$F_s$ Fator de segurança.


$
E_A = 0,5 \cdot (M_1 + M_2) \cdot V_A^2 \cdot C_e \cdot C_a \cdot F_s
$

$M_1$ Massa deslocada pelo navio ($w$);
$M_2$ Massa de água adicional;
$V_A$ Velocidade de atracação perpendicular a linha de atracação;
$C_e$ Coeficiente de excentricidade;
$C_a$ Coeficiente de amortecimento;
$F_s$ Fator de segurança.

## Atracação

### Energia cinética na estrutura
PIANC 2002

$
E_n = 0,5 \cdot M_D \cdot V_a^2 \cdot C_m \cdot C_e \cdot C_s \cdot C_c
$

$E_n$ é a energia cinética característica nominal; 
$M_D$ é o deslocamento da embarcação;
$V_a$ é a velocidade de atracação perpendicular a linha de atracação;
$C_m$ é o coeficiente adicional de massa;
$C_e$ é o coeficiente de excentricidade;
$C_s$ é o coeficiente de rigidez;
$C_c$ é o coeficiente de configuração do cais.

## Implementação

In [ ]:
###################################
### PIANC 2002 Energia Cinética ###
###################################

def solver_energia_cinetica_atracacao(
    v=1,
    En = sym.Symbol('En'),
    MD = sym.Symbol('MD'),
    Va = sym.Symbol('Va'),
    Cm = sym.Symbol('Cm'),
    Ce = sym.Symbol('Ce'),
    Cs = sym.Symbol('Cs'),
    Cc = sym.Symbol('Cc')):
    return(sym.solvers.nsolve(
        (0.5 * MD * (Va**2) * Cm * Ce * Cs * Cc) - En
        , v
    ))

#### Coef. Adicional de Massa

##### PIANC 2002

Para $\frac{K_c}{D} \leq 0,1$

$
C_m = 1,8
$

Para $ 0,1 \leq \frac{K_c}{D} \geq 0,5$

$
C_m = 1,875 - 0,75 \cdot \frac{K_c}{D}
$

Para $\frac{K_c}{D} \geq 0,5$

$
C_m = 1,5
$

##### SHIGERA UEDA 1981

$
C_m = 1 + \frac{\pi \cdot D}{2 \cdot C_B \cdot B}
$

##### VASCO COSTA 1964

$
C_m = 1 + \frac{2 \cdot D}{B}
$

## Implementação

In [ ]:
###################################
#### Coeficiente adc. de massa ####
###################################

def solver_coef_adicional_massa_pianc2002(Kc, D):
    # para Kc/D menor que 0.1
    Cm = 1.8
    # relacao entre Kc e D
    aux = Kc/D
    if(aux >= 0.1 and aux <= 0.5):
        Cm = 1.875 -(0.75 * (Kc/D))
    if(aux >= 0.5):
        Cm = 1.5
    return(Cm)

def solver_coef_adicional_massa_shigeraueda1981(D, CB, B):
    Cm = 1 + ((math.pi * D)/(2 * CB * B))
    return(Cm)

def solver_coef_adicional_massa_vascocosta1964(D, B):
    Cm = 1 + (2*D/B)
    return(Cm)

#### Coef. de Excentricidade

$
C_E = \frac{k^2 + (R^2 \cdot \cos^2(\gamma))}{k^2 + R^2}
$

$
k = (0,19 \cdot C_B + 0,11) \cdot L_{BP}
$

$
R = \sqrt{ \left ( \frac{L_{BP}}{2} - x \right ) ^{2} + \left ( \frac{B}{2} \right ) ^{2} }
$

$
\gamma = 90º - \alpha - \sin^{-1} \left ( \frac{B}{2 \cdot R} \right )
$

## Implementação

In [ ]:
###################################
## Coeficiente de excentricidade ##
###################################

def solver_coef_excentricidade(k, R, gamma):
    Ce = ((k**2)+((R**2) * (math.cos(gamma)**2)))/((k**2)+(R**2))
    return(Ce)

def solver_k(CB, LBP):
    k = (0.19 * CB + 0.11) * LBP
    return(k)

def solver_R(LBP, x, B):
    R = math.sqrt((((LBP/2) - x)**2) + ((B/2)**2))
    return(R)

def solver_gamma(alpha, B, R):
    gamma = 90 - alpha - math.degrees(math.asin(B/(2*R)))
    return(gamma)

#### Coef. de Rigidez

Para

$
\delta_F \leq 150mm \rightarrow C_S \approx 0,9
$

Para

$
\delta_F > 150mm \rightarrow C_S = 1,0
$


## Implementação

In [ ]:
###################################
##### Coeficiente de rigidez ######
###################################

def solver_coef_rigidez(deltaf):
    Cs = 1.0
    # deltaf em milímetros
    if (deltaf <= 150):
        Cs = 0.9
    return(Cs)

### Coef. de Configuração do Cais

##### Estrutura Fechada

Para $ \frac{K_C}{D} \leq 0,5 \rightarrow C_C \approx 0,8 $

Para $ \frac{K_C}{D} > 0,5 \rightarrow C_C \approx 0,9 $

Para $ \alpha > 5° \rightarrow C_C = 1 $

## Implementação

In [ ]:
###################################
### Est. Fechada - C. Conf. Cais ##
###################################

def solver_cc_est_fechada(Kc, D, alpha):
    # relacao entre Kc e D
    aux = Kc/D
    if(aux <= 0.5):
        Cc = 0.8
    if(aux > 0.5):
        Cc = 0.9
    if(alpha > 5):
        Cc = 1
    return(Cc)

##### Estrutura Semi-Fechada

Para $ \frac{K_C}{D} \leq 0,5 \rightarrow C_C \approx 0,9 $

Para $ \frac{K_C}{D} > 0,5 \rightarrow C_C \approx 1,0 $

Para $ \alpha > 5° \rightarrow C_C = 1 $

## Implementação

In [ ]:
###################################
# E. Semi-Fechada - C. Conf. Cais #
###################################

def solver_cc_est_semi_fechada(Kc, D, alpha):
    # relacao entre Kc e D
    aux = Kc/D
    if(aux <= 0.5):
        Cc = 0.9
    if(aux > 0.5):
        Cc = 1.0
    if(alpha > 5):
        Cc = 1
    return(Cc)

##### Estrutura Aberta

$ C_C = 1,0 $

## Implementação

In [ ]:
###################################
### Est. Aberta - C. Conf. Cais ###
###################################

def solver_cc_est_aberta():
    return(1.0)

## Ventos e Correntes na Embarcação

### Força exercida pelo fluido

$
F = C_f \cdot 0,5 \cdot \rho \cdot v^2 \cdot A
$


## Implementação

In [ ]:
###################################
### Força Exercida pelo Fluido ####
###################################

def solver_forca_fluido(
    v=1,
    F = sym.Symbol('F'),
    Cf = sym.Symbol('Cf'),
    rho = sym.Symbol('𝜌'),
    v = sym.Symbol('v'),
    A = sym.Symbol('A')):
    return(sym.solvers.nsolve(
        (Cf * 0.5 * rho * (v**2) * A) - F
        , v
    ))

### Força exercida pelo AR
BSI - 6349

$F_{TW}$ é a força transversal do vento (kN)

$
F_{TW} = C_{TW} \cdot \rho_{A} \cdot A_L \cdot V_w^2 \cdot 10^{-4}
$

$F_{LW}$ é a força longitudinal do vento (kN)

$
F_{LW} = C_{LW} \cdot \rho_{A} \cdot A_L \cdot V_w^2 \cdot 10^{-4}
$

## Implementação

In [ ]:
###################################
##### Força Exercida pelo Ar ######
###################################

def solver_forca_trans_ar(
    v=1,
    FTW = sym.Symbol('FTW'),
    CTW = sym.Symbol('CTW'),
    rhoA = sym.Symbol('𝜌A'),
    AL = sym.Symbol('AL'),
    VW = sym.Symbol('VW')):
    return(sym.solvers.nsolve(
        (CTW * rhoA * AL * (VW**2) * (10**-4)) - FTW
        , v
    ))

def solver_forca_longi_ar(
    v=1,
    FLW = sym.Symbol('FLW'),
    CLW = sym.Symbol('CLW'),
    rhoA = sym.Symbol('𝜌A'),
    AL = sym.Symbol('AL'),
    VW = sym.Symbol('VW')):
    return(sym.solvers.nsolve(
        (CLW * rhoA * AL * (VW**2) * (10**-4)) - FLW
        , v
    ))

### Força exercida pela CORRENTE
BSI - 6349

$F_{TC}$ é a força transversal da corrente (kN)

$
F_{TC} = C_{TC} \cdot C_{CT} \cdot \rho \cdot L_{BP} \cdot d_{m} \cdot V_c^2 \cdot 10^{-4}
$

$F_{LC}$ é a força longitudinal da corrente (kN)

$
F_{LC} = C_{LC} \cdot C_{CL} \cdot \rho \cdot L_{BP} \cdot d_{m} \cdot V_c^2 \cdot 10^{-4}
$

## Implementação

In [ ]:
###################################
## Força Exercida pela Corrente ###
###################################

def solver_forca_trans_corrente(
    v=1,
    FTC = sym.Symbol('FTC'),
    CTC = sym.Symbol('CTC'),
    CCT = sym.Symbol('CCT'),
    rho = sym.Symbol('𝜌'),
    LBP = sym.Symbol('LBP'),
    dm = sym.Symbol('dm'),
    Vc = sym.Symbol('Vc')):
    return(sym.solvers.nsolve(
        (CTC * CCT * rho * LBP * dm * (Vc**2) * (10**-4)) - FTC
        , v
    ))

def solver_forca_longi_corrente(
    v=1,
    FLC = sym.Symbol('FLC'),
    CLC = sym.Symbol('CLC'),
    CCL = sym.Symbol('CCL'),
    rho = sym.Symbol('𝜌'),
    LBP = sym.Symbol('LBP'),
    dm = sym.Symbol('dm'),
    Vc = sym.Symbol('Vc')):
    return(sym.solvers.nsolve(
        (CLC * CCL * rho * LBP * dm * (Vc**2) * (10**-4)) - FLC
        , v
    ))
